In [ ]:
from google.colab import files
import zipfile
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image

uploaded = files.upload()

for fname in uploaded.keys():
  with zipfile.ZipFile(fname, 'r') as zip_ref:
    zip_ref.extractall("dataset")
    print("Dataset extraído em /dataset")

In [ ]:
base_dir = "dataset"

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

val_data = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

history = model.fit(train_data, validation_data=val_data, epochs=10)

In [ ]:
print("Faca upload de uma imagem de gato ou cachorro:")
uploaded = files.upload()

for fname in uploaded.keys():
  img_path = fname
  img = image.load_img(img_path, target_size=(150, 150))

  plt.imshow(img)
  plt.axis("off")
  plt.show()

  img_array = image.img_to_array(img) / 255.0
  img_array = np.expand_dims(img_array, axis=0)

  prediction = model.predict(img_array)

  if prediction[0] > 0.5:
    print("Modelo prevê: **Cachorro**")
  else: 
    print("Modelo prevê: **Gato**")